In [2]:
import flwr as fl
import sys
import numpy as np
from tensorflow import keras
import os
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# class SaveModelStrategy(fl.server.strategy.FedAvg):
#     def aggregate_fit(
#         self,
#         rnd,
#         results,
#         failures
#     ):
#         aggregated_weights = super().aggregate_fit(rnd, results, failures)
#         if aggregated_weights is not None:
#             print(f"Saving round {rnd} aggregated_weights...")
#             np.savez(f"round-{rnd}-weights.npz", *aggregated_weights, allow_pickle=True)
            
#         return aggregated_weights

global_model = keras.Sequential([
            keras.layers.Flatten(input_shape=(28,28)),
            keras.layers.Dense(128, activation='relu'),
            keras.layers.Dense(256, activation='relu'),
            keras.layers.Dense(10, activation='softmax')
        ])
# model = Sequential()
# model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
# model.add(MaxPooling2D((2, 2)))
# model.add(Conv2D(64, (3, 3), activation='relu'))
# model.add(MaxPooling2D((2, 2)))
# model.add(Conv2D(64, (3, 3), activation='relu'))
# model.add(Flatten())
# model.add(Dense(64, activation='relu'))
# model.add(Dense(10, activation='softmax'))


class SaveModelStrategy(fl.server.strategy.FedAvg):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.save_dir = './'

    def aggregate_fit(self, rnd, results, failures):
        aggregated_weights, agg_metrics = super().aggregate_fit(rnd, results, failures)
        if aggregated_weights is not None:
            print(f"Saving round {rnd} aggregated_weights to npz file...") 
            aggregated_ndarrays: List[np.ndarray] = fl.common.parameters_to_ndarrays(aggregated_weights)
            np.savez(f"round-{rnd}-weights.npz", *aggregated_ndarrays)
            global_model.set_weights(aggregated_ndarrays)
        return aggregated_weights, agg_metrics



strategy = SaveModelStrategy()
PORT=5010
fl.server.start_server(
        server_address = 'localhost:'+str(PORT), 
        config=fl.server.ServerConfig(num_rounds=3) ,
        grpc_max_message_length = 1024*1024*1024, strategy = strategy)
        

INFO flower 2023-11-28 23:44:47,344 | app.py:119 | Starting Flower server, config: ServerConfig(num_rounds=3, round_timeout=None)
INFO flower 2023-11-28 23:44:47,371 | app.py:132 | Flower ECE: gRPC server running (3 rounds), SSL is disabled
INFO flower 2023-11-28 23:44:47,372 | server.py:86 | Initializing global parameters
INFO flower 2023-11-28 23:44:47,373 | server.py:270 | Requesting initial parameters from one random client
INFO flower 2023-11-28 23:44:53,865 | server.py:274 | Received initial parameters from one random client
INFO flower 2023-11-28 23:44:53,867 | server.py:88 | Evaluating initial parameters
INFO flower 2023-11-28 23:44:53,868 | server.py:101 | FL starting
DEBUG flower 2023-11-28 23:44:54,862 | server.py:215 | fit_round 1: strategy sampled 2 clients (out of 2)
DEBUG flower 2023-11-28 23:45:45,445 | server.py:229 | fit_round 1 received 2 results and 0 failures
WARNING flower 2023-11-28 23:45:45,485 | fedavg.py:243 | No fit_metrics_aggregation_fn provided


Saving round 1 aggregated_weights to npz file...


DEBUG flower 2023-11-28 23:45:47,850 | server.py:165 | evaluate_round 1: strategy sampled 2 clients (out of 2)
DEBUG flower 2023-11-28 23:45:52,318 | server.py:179 | evaluate_round 1 received 2 results and 0 failures
WARNING flower 2023-11-28 23:45:52,321 | fedavg.py:274 | No evaluate_metrics_aggregation_fn provided
DEBUG flower 2023-11-28 23:45:52,322 | server.py:215 | fit_round 2: strategy sampled 2 clients (out of 2)
DEBUG flower 2023-11-28 23:46:44,511 | server.py:229 | fit_round 2 received 2 results and 0 failures


Saving round 2 aggregated_weights to npz file...


DEBUG flower 2023-11-28 23:46:47,008 | server.py:165 | evaluate_round 2: strategy sampled 2 clients (out of 2)
DEBUG flower 2023-11-28 23:46:54,333 | server.py:179 | evaluate_round 2 received 2 results and 0 failures
DEBUG flower 2023-11-28 23:46:54,354 | server.py:215 | fit_round 3: strategy sampled 2 clients (out of 2)
DEBUG flower 2023-11-28 23:47:51,532 | server.py:229 | fit_round 3 received 2 results and 0 failures


Saving round 3 aggregated_weights to npz file...


DEBUG flower 2023-11-28 23:47:53,880 | server.py:165 | evaluate_round 3: strategy sampled 2 clients (out of 2)
DEBUG flower 2023-11-28 23:48:00,800 | server.py:179 | evaluate_round 3 received 2 results and 0 failures
INFO flower 2023-11-28 23:48:00,808 | server.py:144 | FL finished in 186.9386407
INFO flower 2023-11-28 23:48:00,810 | app.py:180 | app_fit: losses_distributed [(1, 0.4277659058570862), (2, 0.32093989849090576), (3, 0.2257644236087799)]
INFO flower 2023-11-28 23:48:00,812 | app.py:181 | app_fit: metrics_distributed {}
INFO flower 2023-11-28 23:48:00,813 | app.py:182 | app_fit: losses_centralized []
INFO flower 2023-11-28 23:48:00,814 | app.py:183 | app_fit: metrics_centralized {}


History (loss, distributed):
	round 1: 0.4277659058570862
	round 2: 0.32093989849090576
	round 3: 0.2257644236087799

In [11]:
global_model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_3 (Flatten)         (None, 784)               0         
                                                                 
 dense_9 (Dense)             (None, 128)               100480    
                                                                 
 dense_10 (Dense)            (None, 256)               33024     
                                                                 
 dense_11 (Dense)            (None, 10)                2570      
                                                                 
Total params: 136,074
Trainable params: 136,074
Non-trainable params: 0
_________________________________________________________________


In [12]:
global_model.get_weights()

[array([[-0.06576853, -0.03796841, -0.03208628, ...,  0.0361251 ,
         -0.05090229,  0.00166951],
        [-0.03291868, -0.04695641,  0.07912853, ...,  0.08003717,
          0.08009898, -0.04726976],
        [ 0.02718371, -0.05731468, -0.01467272, ..., -0.04320089,
         -0.00837144,  0.0511827 ],
        ...,
        [ 0.06960624, -0.0214816 , -0.01851103, ...,  0.0710083 ,
          0.00209894, -0.05163113],
        [-0.06784113,  0.06735349,  0.03802332, ..., -0.06084887,
         -0.07804901, -0.07424384],
        [-0.03585071, -0.00941872,  0.0608483 , ..., -0.01503462,
         -0.03089369,  0.00177036]], dtype=float32),
 array([ 0.04866719, -0.00866843,  0.0155193 ,  0.03632028,  0.02947935,
        -0.03273496,  0.02243583, -0.02575295,  0.0187433 ,  0.01481763,
        -0.01861057,  0.04988173,  0.03360007,  0.07139365, -0.00533522,
         0.04732795, -0.04423813, -0.03461048, -0.03922785, -0.00259434,
        -0.0467128 ,  0.01691433,  0.0304389 ,  0.05195938,  0.014

In [13]:
import tensorflow as tf

global_model.compile("adam", "sparse_categorical_crossentropy", metrics=["accuracy"])
(_,_), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_test = x_test/255.0

x_test = x_test.reshape(x_test.shape[0],28,28,1)
acc = global_model.evaluate(x_test, y_test)

print(acc)

313/313 [==============================] - 2s 5ms/step - loss: 0.1554 - accuracy: 0.9497
[0.15535546839237213, 0.9496999979019165]
